# Longtype
* 차량 쌍(TV, LV) 기준으로 정리하기
* 왜냐하면, 각 쌍별로 SSM이 계산되기 때문임

# Import

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import warnings
import os

from scipy import stats # Z-score를 이용한 이상값 제거

import math # arctangent; math.atan 사용 목적
import statistics

import pyarrow.parquet as pq

# Load Dataset

In [2]:
working_dir = 'D:/OneDrive/Projects/2023_SSM_Feasibility/Dataset'

In [3]:
data_folder = '02_processed'
data_name = 'LV_merging_type1_03frm_0.10sec.parquet'
data_path = os.path.join(working_dir, data_folder, data_name)

In [4]:
df_1 =pq.read_pandas(data_path).to_pandas()
df_1.rename({'Vehicle ID' : 'veh_id', 'frame' : 'frm',
             'Local X (m)_before' : 'local_x_before',
             'Local Y(m)_before' : 'local_y_before',
             'Lane Identification' : 'lane',
             'Lane Identification Past' : 'lane_past',
             'LV0_lane_record' : 'LV0_lane_record', 
             'LV0_lane_record_split' : 'LV0_Lane_record_split',
             'LV0_lane_change_direction' : 'LV0_Lane_change_direction',
             'LVL_lane_record' : 'LVL_Lane_record',
             'LVL_lane_record_split' : 'LVL_Lane_record_split',
             'LVL_lane_change_direction' : 'LVL_Lane_change_direction',
             'LVR_lane_record' : 'LVR_Lane_record',
             'LVR_lane_record_split' : 'LVR_Lane_record_split',
             'LVR_lane_change_direction' : 'LVR_Lane_change_direction',
            }, axis = 1, inplace = True)

In [5]:
df_1.head(3)

,veh_id,frm,Total Frames,Global Time (Epoch Time),local_x,local_y,V_len,V_wid,Vehicle Class,velocity,...,LVR_delta_velocity_y,LVR_acc_x,LVR_acc_y,LVR_Lane_record,LVR_Lane_record_split,LVR_Lane_change_direction,LV0_D,LVL_D,LVR_D,LC_CF
index,,,,,,,,,,,,,,,,,,,,,
0,1,339,39,46811300,51.06250,10.617188,5.289062,2.193359,2,87.6250,...,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,CF
1,1,342,42,46811400,53.53125,10.640625,5.289062,2.193359,2,89.0625,...,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,CF
2,1,345,45,46811500,56.00000,10.648438,5.289062,2.193359,2,88.2500,...,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,CF


# Longtype
* pairs 리스트의 튜플(TV, LV)에 따라서 각 차량의 위치, 속도, 가속도 등을 df로부터 프레임별로 뽑아오기

In [6]:
for df, i in tqdm(zip([df_1], [1])):

    df_LV0 = df[['veh_id', 'LV0_ID', 'frm']].copy()
    df_LV0 = df_LV0.rename({'LV0_ID' : 'LV_ID'}, axis = 1)
    df_LV0['LV_type'] = 'LV0'
    
    df_LVL = df[['veh_id', 'LVL_ID', 'frm']].copy()
    df_LVL = df_LVL.rename({'LVL_ID' : 'LV_ID'}, axis = 1)
    df_LVL['LV_type'] = 'LVL'

    df_LVR = df[['veh_id', 'LVR_ID', 'frm']].copy()
    df_LVR = df_LVR.rename({'LVR_ID' : 'LV_ID'}, axis = 1)
    df_LVR['LV_type'] = 'LVR'
    
    total_long = pd.concat([df_LV0, df_LVL, df_LVR])
    
    # LV_ID가 0인 것은 제거한다
    total_long = total_long[(total_long['LV_ID'] != 0) & (pd.isna(total_long['LV_ID']) == False)]
    total_long['LV_ID'] = total_long['LV_ID'].astype('int')
    total_long.reset_index(inplace = True, drop = True)
    
    globals()[f'long_{i}'] = total_long

1it [00:00, 71.75it/s]


In [60]:
veh_pair = long_1[['veh_id', 'LV_ID']].drop_duplicates()

In [61]:
veh_pair

,veh_id,LV_ID
0,2,1
13,4,2
25,5,4
84,8,2
102,10,6
...,...,...
65251,1616,1614
65299,1618,1617
65326,1618,1620
65373,1626,1629


In [55]:
len(reference_df)

47157

In [62]:
long_1

,veh_id,LV_ID,frm,LV_type
0,2,1,540,LV0
1,2,1,543,LV0
2,2,1,546,LV0
3,2,1,549,LV0
4,2,1,552,LV0
...,...,...,...,...
65416,1631,1629,26949,LVR
65417,1631,1629,26952,LVR
65418,1631,1629,26955,LVR
65419,1631,1629,26958,LVR


In [76]:
def make_int(x):
    if pd.isna(x) == False:
        return int(x)

    else:
        return None

In [94]:
warnings.filterwarnings('ignore')

target_df_cols = ['local_x', 'local_y', 'V_len', 'V_wid', 'velocity', 'acc', 'lane', 'Time',
                  'local_x_before', 'local_y_before', 'delta_local_x', 'delta_local_y',
                  'velocity_x', 'velocity_y', 'velocity_x_before', 'velocity_y_before', 'delta_velocity_x', 'delta_velocity_y',
                  'acc_x', 'acc_y', 'lane_past', 'Lane_record', 'Lane_record_split', 'Lane_00', 'Lane_99',
                  'Lane_change', 'Lane_leave', 'Lane_change_direction', 'LC_CF']

save_folder = '02_processed_long'
save_files = ['merging_type1_03frm_0.10sec.parquet']


for long, i in zip([long_1], [1]):

    # 레퍼런스 데이터프레임
    reference_df = globals()[f'df_{i}'][['veh_id', 'frm'] + target_df_cols].copy() # Merge의 reference가 될 데이터프레임
    veh_list = list(reference_df['veh_id'].unique())

    long_tot = pd.DataFrame()
    
    for i in tqdm(range(len(veh_pair))): # 각 차량에 대하여
        row = veh_pair.iloc[i] # 차량 쌍
        veh_id = row['veh_id']
        LV_ID = row['LV_ID']

        veh_TV = reference_df[reference_df['veh_id'] == veh_id].copy()
        veh_LV = reference_df[reference_df['veh_id'] == LV_ID].copy()

        # LV 데이터프레임 컬럼이름을 적절하게 변경
        for col in target_df_cols:
            veh_LV = veh_LV.rename({col : 'LV_'+col}, axis = 1)

        veh_TV = pd.merge(veh_TV, long, how = 'left', left_on = ['veh_id', 'frm'], right_on = ['veh_id', 'frm'])
        veh_tot = pd.merge(veh_TV, veh_LV, how = 'left', left_on = ['LV_ID', 'frm'], right_on = ['veh_id', 'frm']).rename({'veh_id_x' : 'veh_id'}, axis = 1).drop(['veh_id_y'], axis = 1)

        long_tot = pd.concat([long_tot, veh_tot])

    long_tot = long_tot.reset_index(drop = True).sort_values(by = ['veh_id', 'LV_ID', 'frm'])

    #long_tot = pd.concat([long_TV, long_LV], axis = 1)

    # 거리관련 변수
    long_tot['D_x'] = long_tot['LV_local_x'] - long_tot['local_x']
    long_tot['D_y'] = long_tot['LV_local_y'] - long_tot['local_y']
    long_tot['D'] = (long_tot['D_x']**2 + long_tot['D_y']**2) ** (1/2)
    long_tot.reset_index(inplace = True, drop = True)
    
    long_tot['D_lag'] = long_tot['D'].shift(1) # 직전 row의 값임
    long_tot['veh_id'] = long_tot['veh_id'].astype('int')
    long_tot['LV_ID'] = long_tot['LV_ID'].apply(make_int)
    long_tot['frm'] = long_tot['frm'].astype('int')
    
    long_tot['veh_id_lag'] = long_tot['veh_id'].shift(1) # 직전 row 값
    long_tot['LV_ID_lag'] = long_tot['LV_ID'].shift(1) # 직전 row 값
    
    long_tot['D_gap'] = long_tot['D'] - long_tot['D_lag'] # 거리 변화. (-)이면 줄어들고 있다는 뜻이다
    
    # 만약 veh_id, LV_id 의 lag값이 현재 row와 다르다면, 새로운 값이 시작된 것이므로 D_gap을 NaN으로 만들어줘야 한다.
    long_tot.loc[(long_tot['veh_id'] != long_tot['veh_id_lag']) | (long_tot['LV_ID'] != long_tot['LV_ID_lag']), 'D_gap'] = None
        
    #print(f'{i} : {len(long_final)}')
    long_tot.reset_index(inplace = True, drop = True)
    long_tot.drop(['D_lag', 'veh_id_lag', 'LV_ID_lag'], axis = 1, inplace = True)
    
    # 저장하기
    save_path = os.path.join(working_dir, save_folder, save_files[0])
    long_tot.to_parquet(save_path, engine = 'fastparquet', compression = 'gzip') 

100%|██████████████████████████████████████████████████████████████████████████████| 2005/2005 [01:24<00:00, 23.70it/s]


In [102]:
veh_id = 1544
LV_ID = 1542

In [105]:
veh_TV = reference_df[reference_df['veh_id'] == veh_id].copy()
veh_LV = reference_df[reference_df['veh_id'] == LV_ID].copy()

In [107]:
# LV 데이터프레임 컬럼이름을 적절하게 변경
for col in target_df_cols:
    veh_LV = veh_LV.rename({col : 'LV_'+col}, axis = 1)

In [109]:
veh_TV.head(3)

,veh_id,frm,local_x,local_y,V_len,V_wid,velocity,acc,lane,Time,...,acc_y,lane_past,Lane_record,Lane_record_split,Lane_00,Lane_99,Lane_change,Lane_leave,Lane_change_direction,LC_CF
index,,,,,,,,,,,,,,,,,,,,,
45123,1544,25572,47.12500,6.910156,4.902344,2.193359,131.250,0.254150,U2,852.4,...,NaN,U2,U2,U2_U2,U2,U2,None,None,Straight,CF
45124,1544,25575,50.78125,6.964844,4.902344,2.193359,132.125,2.285156,U2,852.5,...,NaN,U2,U2,U2_U2,U2,U2,None,None,Straight,CF
45125,1544,25578,54.43750,7.027344,4.902344,2.193359,131.125,-2.505859,U2,852.6,...,0.051849,U2,U2,U2_U2,U2,U2,None,None,Straight,CF


In [108]:
veh_LV

,veh_id,frm,LV_local_x,LV_local_y,LV_V_len,LV_V_wid,LV_velocity,LV_acc,LV_lane,LV_Time,...,LV_acc_y,LV_lane_past,LV_Lane_record,LV_Lane_record_split,LV_Lane_00,LV_Lane_99,LV_Lane_change,LV_Lane_leave,LV_Lane_change_direction,LV_LC_CF
index,,,,,,,,,,,,,,,,,,,,,
45000,1542,25518,37.56250,12.539062,7.609375,2.193359,94.3125,1.446289,U4,850.6,...,NaN,U4,U4_U3_U2,U4_U3,U4,U2,None,None,Left,LC
45001,1542,25521,40.21875,12.437500,7.609375,2.193359,94.9375,1.681641,U4,850.7,...,NaN,U4,U4_U3_U2,U4_U3,U4,U2,None,None,Left,LC
45002,1542,25524,42.84375,12.312500,7.609375,2.193359,94.5625,-1.055664,U4,850.8,...,-0.153076,U4,U4_U3_U2,U4_U3,U4,U2,None,None,Left,LC
45003,1542,25527,45.46875,12.171875,7.609375,2.193359,95.3125,2.205078,U4,850.9,...,-0.160522,U4,U4_U3_U2,U4_U3,U4,U2,None,None,Left,LC
45004,1542,25530,48.15625,12.062500,7.609375,2.193359,96.3125,2.750000,U4,851.0,...,0.237671,U4,U4_U3_U2,U4_U3,U4,U2,None,None,Left,LC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45067,1542,25719,221.37500,10.593750,7.609375,2.193359,99.3750,4.152344,U2,857.3,...,0.158569,U2,U4_U3_U2,U2_U2,U4,U2,None,None,Straight,CF
45068,1542,25722,224.12500,10.640625,7.609375,2.193359,98.0000,-3.708984,U2,857.4,...,-0.142700,U2,U4_U3_U2,U2_U2,U4,U2,None,None,Straight,CF
45069,1542,25725,226.75000,10.679688,7.609375,2.193359,98.1250,0.299805,U2,857.5,...,-0.057922,U2,U4_U3_U2,U2_U2,U4,U2,None,None,Straight,CF
